In [9]:
#!pip install dbdpy
#!pip install dbfread
#!pip install pandas


  Using cached pandas-2.2.2-cp310-cp310-win_amd64.whl.metadata (19 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ----------- ---------------------------- 3.4/11.6 MB 22.3 MB/s eta 0:00:01
   -------------------------------- ------- 9.4/11.6 MB 32.7 MB/s eta 0:00:01
   ------------------------------------ --- 10.5/11.6 MB 18.7 MB/s eta 0:00:01
   ---------------------------------------  11.5/11.6 MB 15.0 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 13.7 MB/s eta 0:00:00


In [10]:
import os
import pandas as pd
from dbfread import DBF

In [36]:
#Especificamos la ruta de los archivos DFB
CARPETA_PALTA = "D:/Proyectos/Palta/Datos"

#creamos una lista en donde se van a almacenar los dataframes
dataframes = []

#Recorremos los archivos de la carpeta
for archivo in os.listdir(CARPETA_PALTA):
    if archivo.endswith(".DBF"):
        #leemos el archivo
        rutaArchivo = os.path.join(CARPETA_PALTA, archivo)        
        tabla = DBF(rutaArchivo, encoding = "latin1")
        df = pd.DataFrame(iter(tabla))
        dataframes.append(df)


#Concatena todos los Dataframes en uno solo
df_concatenado = pd.concat(dataframes, ignore_index = True)





                                                    frecuencia  Porcentaje  \
DESC_COM                                                                     
PALTAS                                                    1444    1.210475   
PALTAS FRESCAS EN CAJAS / VARIEDAD : HASS SE AC...        1059    0.887738   
PALTA  HASS                                                811    0.679844   
PALTA FRESCA HASS                                          665    0.557456   
PALTAS FRESCAS VARIEDAD HASS EN CAJAS PARA CONS...         653    0.547396   
...                                                        ...         ...   
PALTAS FRESCAS VARIEDAD HASS EN CAJAS X 4 Y 10 ...           1    0.000838   
PALTAS FRESCAS EN CAJAS, PLASTICO 10 KG. DOLE /...           1    0.000838   
PALTAS FRESCAS EN CAJAS, VARIEDAD: HASS/CALIBRE...           1    0.000838   
PALTAS FRESCAS EN CAJAS, VARIEDAD: HASS/CALIBRE...           1    0.000838   
PALTA VARIEDAD "HASS" CALIBRE 10,12,14,16,18,20...           1  

In [42]:
df_concatenado.describe()


,FECHA,FOB_DOLPOL,PESO_NETO,PESO_BRUTO,UNID_FIQTY,FEC_NUM,FEC_REG
count,1.192920e+05,119292.000000,119292.000000,119292.000000,119292.000000,1.192920e+05,1.192920e+05
mean,2.016370e+07,38375.111416,20254.040933,22383.803541,20254.319376,2.016368e+07,1.986674e+07
std,4.108077e+04,18572.379470,7558.885325,8604.683803,7559.047782,4.108032e+04,2.426548e+06
min,2.001013e+07,0.000000,0.000000,0.000000,0.000000,2.001013e+07,0.000000e+00
25%,2.014063e+07,26936.000000,20400.000000,22620.000000,20400.000000,2.014062e+07,2.014070e+07
50%,2.017070e+07,38696.000000,21120.000000,23731.000000,21120.000000,2.017063e+07,2.017070e+07
75%,2.019061e+07,48713.000000,22080.000000,24420.000000,22080.000000,2.019060e+07,2.019061e+07
max,2.024090e+07,376596.000000,248976.000000,718272.000000,248976.000000,2.024090e+07,2.024091e+07


In [39]:
#Análisis para Limpieza de datos
"""#FrecuenciaTexto
frecuencia = df_concatenado["DESC_COM"].value_counts()
#Porcentaje de texto
porcentajeDeFrecuencia = (frecuencia / frecuencia.sum())*100

#Creamos un dataframe con los resultados
resultados = pd.DataFrame({"frecuencia": frecuencia, "Porcentaje": porcentajeDeFrecuencia} )
#Calculamos el porcentaje acumulado
resultados["Porcentaje acumulado"] = resultados["Porcentaje"].cumsum()
#Filtramos los que llegan al 80%

resultados= resultados[resultados["Porcentaje acumulado"]>80]
print(resultados)"""

'#FrecuenciaTexto\nfrecuencia = df_concatenado["DESC_COM"].value_counts()\n#Porcentaje de texto\nporcentajeDeFrecuencia = (frecuencia / frecuencia.sum())*100\n\n#Creamos un dataframe con los resultados\nresultados = pd.DataFrame({"frecuencia": frecuencia, "Porcentaje": porcentajeDeFrecuencia} )\n#Calculamos el porcentaje acumulado\nresultados["Porcentaje acumulado"] = resultados["Porcentaje"].cumsum()\n#Filtramos los que llegan al 80%\n\nresultados= resultados[resultados["Porcentaje acumulado"]>80]\nprint(resultados)'

In [31]:
df["DESC_COM"].unique

<bound method Series.unique of 0                PALTAS FRESCAS
1      PULPA DE PALTA CONGELADA
2                PALTAS FRESCAS
3           PALTAS REFRIGERADAS
4           PALTAS REFRIGERADAS
                 ...           
99          PALTAS REFRIGERADAS
100         PALTAS REFRIGERADAS
101         PALTAS REFRIGERADAS
102         PALTAS REFRIGERADAS
103              PALTOS FRESCOS
Name: DESC_COM, Length: 104, dtype: object>